External Validation – Hospital Dataset
Model trained on Sleep-EDF Expanded


In [5]:
import mne
import os
import numpy as np
import glob
from sklearn.model_selection import train_test_split

# ==========================================
# 1. הגדרות נתיבים ופרמטרים
# ==========================================
# שמי לב לעדכן את הנתיב המלא לתיקייה שלך
RAW_FOLDER = '/Users/talhakimi/Desktop/יישומים רפואיים/deepsleep/sleep-edf-database-expanded-1.0.0'
OUTPUT_FILE = 'hospital_st_processed.npz'

CHANNELS = ['EEG Fpz-Cz', 'EEG Pz-Oz', 'EOG horizontal']
SFREQ = 100 

# מיפוי שלבי שינה (איחוד N3 ו-N4 לפי תקן AASM)
stage_mapping = {
    'Sleep stage W': 0, 'Sleep stage 1': 1, 'Sleep stage 2': 2,
    'Sleep stage 3': 3, 'Sleep stage 4': 3, 'Sleep stage R': 4
}

# איתור קבצים בכל התת-תיקיות
psg_files = sorted(glob.glob(os.path.join(RAW_FOLDER, '**', '*PSG.edf'), recursive=True))
hypno_files = sorted(glob.glob(os.path.join(RAW_FOLDER, '**', '*Hypnogram.edf'), recursive=True))

hospital_x, hospital_y = [], []

print(f"🚀 Found {len(psg_files)} total files. Filtering for Hospital (ST) subjects...")

# ==========================================
# 2. עיבוד נתוני בית החולים (ST)
# ==========================================
for psg_p in psg_files:
    subject_id = os.path.basename(psg_p)[:6]
    
    # סינון: לוקחים רק נבדקי ST (בית חולים)
    if 'ST' not in subject_id:
        continue 
    
    # מציאת ה-Hypnogram המתאים
    hypno_p = [h for h in hypno_files if subject_id in os.path.basename(h)]
    if not hypno_p:
        continue
    hypno_p = hypno_p[0]

    try:
        # טעינת אותות וסימונים
        raw = mne.io.read_raw_edf(psg_p, preload=True, verbose=False)
        annot = mne.read_annotations(hypno_p)
        raw.set_annotations(annot, emit_warning=False)
        
        # בחירת ערוצים ושינוי תדר דגימה
        raw.pick([c for c in CHANNELS if c in raw.ch_names])
        if raw.info['sfreq'] != SFREQ:
            raw.resample(SFREQ)

        # קיצוץ ערנות עודפת (30 דקות לפני ואחרי שינה)
        desc = [str(d).upper() for d in annot.description]
        valid_idx = [i for i, d in enumerate(desc) if any(s in d for s in ['1','2','3','4','R'])]
        
        if valid_idx:
            tmin = max(0, annot.onset[valid_idx[0]] - 30 * 60)
            tmax = min(raw.times[-1], annot.onset[valid_idx[-1]] + 30 * 60)
            raw.crop(tmin=tmin, tmax=tmax)
        
        # יצירת Epochs של 30 שניות
        events, event_id = mne.events_from_annotations(raw, event_id=stage_mapping, chunk_duration=30., verbose=False)
        epochs = mne.Epochs(raw, events, event_id, tmin=0, tmax=29.99, baseline=None, preload=True, verbose=False, on_missing='ignore')
        
        if len(epochs) == 0: continue

        X_sub = epochs.get_data() # (Epochs, Channels, Time)

        # נרמול Z-score לכל ערוץ בנפרד
        for ch in range(X_sub.shape[1]):
            mean, std = np.mean(X_sub[:, ch, :]), np.std(X_sub[:, ch, :])
            X_sub[:, ch, :] = (X_sub[:, ch, :] - mean) / (std + 1e-6)

        # סידור ממדים ל-(Epochs, Time, Channels) ושמירה ב-float16 לחיסכון בזיכרון
        X_sub = np.transpose(X_sub, (0, 2, 1))
        hospital_x.append(X_sub.astype(np.float16))
        hospital_y.append(epochs.events[:, 2])

        print(f"✅ Hospital Subject {subject_id} processed ({len(epochs)} epochs).")

    except Exception as e:
        print(f"❌ Error in {subject_id}: {e}")

# ==========================================
# 3. איחוד ושמירה
# ==========================================
if hospital_x:
    X_final = np.concatenate(hospital_x, axis=0).astype(np.float32)
    y_final = np.concatenate(hospital_y, axis=0)

    print(f"\n📊 Total Hospital Data: {X_final.shape}")
    print(f"Distribution: {np.bincount(y_final)}")

    np.savez(OUTPUT_FILE, x=X_final, y=y_final)
    print(f"💾 Saved to {OUTPUT_FILE}")
else:
    print("❌ No hospital data was processed. Check your paths!")

🚀 Found 197 total files. Filtering for Hospital (ST) subjects...


/var/folders/db/xl8268ms1js3ff3nv7003flh0000gn/T/ipykernel_2723/1726900323.py:49: RuntimeWarning: Channels contain different highpass filters. Highest filter setting will be stored.
  raw = mne.io.read_raw_edf(psg_p, preload=True, verbose=False)
/var/folders/db/xl8268ms1js3ff3nv7003flh0000gn/T/ipykernel_2723/1726900323.py:49: RuntimeWarning: Channels contain different lowpass filters. Lowest filter setting will be stored.
  raw = mne.io.read_raw_edf(psg_p, preload=True, verbose=False)


✅ Hospital Subject ST7011 processed (1092 epochs).


/var/folders/db/xl8268ms1js3ff3nv7003flh0000gn/T/ipykernel_2723/1726900323.py:49: RuntimeWarning: Channels contain different highpass filters. Highest filter setting will be stored.
  raw = mne.io.read_raw_edf(psg_p, preload=True, verbose=False)
/var/folders/db/xl8268ms1js3ff3nv7003flh0000gn/T/ipykernel_2723/1726900323.py:49: RuntimeWarning: Channels contain different lowpass filters. Lowest filter setting will be stored.
  raw = mne.io.read_raw_edf(psg_p, preload=True, verbose=False)


✅ Hospital Subject ST7012 processed (1040 epochs).


...

✅ Hospital Subject ST7222 processed (951 epochs).


/var/folders/db/xl8268ms1js3ff3nv7003flh0000gn/T/ipykernel_2723/1726900323.py:49: RuntimeWarning: Channels contain different highpass filters. Highest filter setting will be stored.
  raw = mne.io.read_raw_edf(psg_p, preload=True, verbose=False)
/var/folders/db/xl8268ms1js3ff3nv7003flh0000gn/T/ipykernel_2723/1726900323.py:49: RuntimeWarning: Channels contain different lowpass filters. Lowest filter setting will be stored.
  raw = mne.io.read_raw_edf(psg_p, preload=True, verbose=False)


✅ Hospital Subject ST7241 processed (1021 epochs).


/var/folders/db/xl8268ms1js3ff3nv7003flh0000gn/T/ipykernel_2723/1726900323.py:49: RuntimeWarning: Channels contain different highpass filters. Highest filter setting will be stored.
  raw = mne.io.read_raw_edf(psg_p, preload=True, verbose=False)
/var/folders/db/xl8268ms1js3ff3nv7003flh0000gn/T/ipykernel_2723/1726900323.py:49: RuntimeWarning: Channels contain different lowpass filters. Lowest filter setting will be stored.
  raw = mne.io.read_raw_edf(psg_p, preload=True, verbose=False)


✅ Hospital Subject ST7242 processed (893 epochs).

📊 Total Hospital Data: (42362, 3000, 3)
Distribution: [ 4154  3653 19791  6415  8349]
💾 Saved to hospital_st_processed.npz


In [1]:
import numpy as np
import torch
from imblearn.over_sampling import SMOTE
from torch.utils.data import DataLoader

# 1. טעינת נתוני ה-Sleep-EDF (בהנחה שהם כבר ב-NPZ)
# אם הם בפורמט EDF גולמי, יש להשתמש ב-MNE כדי להמיר אותם קודם
data_edf = np.load('sleepedf_data.npz') 
X_st = data_edf['x']  # מבנה: (Epochs, Time, Channels)
y_st = data_edf['y']

# 2. הכנה ל-SMOTE: שיטוח הנתונים (Flattening)
# SMOTE לא יודע לקרוא מטריצות תלת-ממדיות, לכן נהפוך כל Epoch לוקטור ארוך
n_epochs, n_time, n_channels = X_st.shape
X_flat = X_st.reshape(n_epochs, -1)

# 3. יצירת דגימות סינתטיות (Oversampling)
# זה חשוב במיוחד ב-Sleep-EDF כי שלב N1 הוא מאוד דל בנתונים
print(f"Original class distribution: {np.bincount(y_st)}")
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X_flat, y_st)
print(f"Balanced class distribution: {np.bincount(y_res)}")

# 4. החזרה למבנה המקורי של המודל
X_res = X_res.reshape(-1, n_time, n_channels)

# 5. הגדרת ה-Dataset וה-Loader לבדיקה
# נשתמש ב-SleepSequenceDataset שהגדרת קודם
test_dataset = SleepSequenceDataset(X_res, y_res, seq_length=5)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# 6. טעינת המודל והרצה (Inference)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SleepNetSeq().to(device)
model.load_state_dict(torch.load('model_ros11.pth'))
model.eval()

# 7. הערכת ביצועים
all_preds = []
all_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.numpy())

# חישוב דיוק סופי
accuracy = np.mean(np.array(all_preds) == np.array(all_labels))
print(f"Accuracy on Balanced Sleep-EDF: {accuracy * 100:.2f}%")

FileNotFoundError: [Errno 2] No such file or directory: 'sleepedf_data.npz'